In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import misc

In [7]:
from keras.applications.mobilenet import MobileNet
from keras.layers import Input,Dense,Dropout,Lambda
from keras.models import Model
from keras import backend as K

In [9]:
# Loading the training data as numpy arrays
training_set = pd.read_csv("fashion-mnist_train.csv", dtype='int')
X_train = training_set.drop('label', axis = 1).as_matrix()
Y_train = training_set['label'].as_matrix()

dim = 28

In [10]:
# Preprocessing data: doubling the dimensions since they are 28px (< 32)
#   and hence do not have valid input shape for the MobileNet model
def double_dimensions(images, height, width=-1):
    '''
    double_dimensions(images, height, width) doubles the size
        of the image data in the numpy array images
    height and width must be the current dimensions of the images
    If width is omitted, the data is treated as square images, i.e.
        width = height
    '''
    if width == -1:
        width = height

    # reshaping data to actual image dimensions and doubling the size
    width_x2 = width*2
    height_x2 = height*2
    resized_images = [misc.imresize(img.reshape(height, width), (height_x2,width_x2)).astype(float)
                      for img in images]
    
    return np.array(resized_images)/255

X_train = double_dimensions(X_train, dim)

In [11]:
# Training the model

# The Input Layer: accepts 56 by 56 images
input_image = Input(shape=(dim*2,dim*2))
# Expanding the dimensionality
input_image_dim = Lambda(lambda x: K.repeat_elements(K.expand_dims(x,3),3,3))(input_image)

# Using MobileNet as the pre-trained base model
base_model = MobileNet(input_tensor=input_image_dim, include_top=False, pooling='avg')
# Randomly dropping 1/2 of the input units to prevent overfitting
output = Dropout(0.5)(base_model.output)
# Adding a logistic layer for 10 classes
predict = Dense(10, activation='softmax')(output)

# the final model to train
model = Model(inputs=input_image, outputs=predict)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 56, 56)            0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 56, 56, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 28, 28, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 28, 28, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 28, 28, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 28, 28, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 28, 28, 32)        128       
__________

In [12]:
model.fit(X_train, Y_train, batch_size=64, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 2090s - loss: 0.4188 - acc: 0.8670  
Epoch 2/10
60000/60000 [==============================] - 2064s - loss: 0.2478 - acc: 0.9131  
Epoch 3/10
60000/60000 [==============================] - 2042s - loss: 0.2178 - acc: 0.9247  
Epoch 4/10
60000/60000 [==============================] - 2040s - loss: 0.1889 - acc: 0.9347  
Epoch 5/10
60000/60000 [==============================] - 2065s - loss: 0.1749 - acc: 0.9386  
Epoch 6/10
60000/60000 [==============================] - 2064s - loss: 0.1563 - acc: 0.9456  
Epoch 7/10
60000/60000 [==============================] - 2041s - loss: 0.1562 - acc: 0.9456  
Epoch 8/10
60000/60000 [==============================] - 2084s - loss: 0.1462 - acc: 0.9507  
Epoch 9/10
60000/60000 [==============================] - 2062s - loss: 0.1227 - acc: 0.9577  
Epoch 10/10
60000/60000 [==============================] - 2063s - loss: 0.1070 - acc: 0.9632  


In [15]:
# Predicting labels on the test set
test_set = pd.read_csv("test_data.csv", index_col=0, dtype='int')
X_test = test_set.as_matrix()
X_test = double_dimensions(X_test, dim)

In [16]:
Y_probs = model.predict(X_test)

In [17]:
Y_predicted = Y_probs.argmax(axis=-1)

In [22]:
# Creating the submission
mn_sel_submit = pd.DataFrame({'ids': [i for i in range(10000)],
                             'label': Y_predicted}, dtype=int)
mn_sel_submit.to_csv('mn_predictions.csv', index=False)